In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

from sklearn import decomposition
from sklearn import manifold
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import copy
from collections import namedtuple
import os, re, time
import random
import shutil
import math

import pandas as pd
from sklearn import svm

# set seed to make sure the results are reproducible
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# change directory of data
CV_ID = 1
CV_ID = str(CV_ID)
datapath = os.path.join(".data","hcb","cv" + CV_ID)
save_name='basic-model-cv'+CV_ID+'.pt'
train_dir = os.path.join(datapath, 'train')
val_dir = os.path.join(datapath, 'val')
test_dir = os.path.join(datapath, 'test')

pretrained_size = (224, 224)
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                        #    transforms.RandomRotation(50),
                           transforms.RandomRotation(30),
                           transforms.RandomHorizontalFlip(0.4),
                           transforms.RandomVerticalFlip(0.4),
                           transforms.RandomCrop(pretrained_size, padding = 10),
                           transforms.CenterCrop(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                       ])
# inorder to load img with it's label
class MyImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, _ = self.imgs[index] #img path, label
        return super(MyImageFolder, self).__getitem__(index), path # return image path
    

BATCH_SIZE = 8#4
train_data = MyImageFolder(root = train_dir,
                                transform = train_transforms)
test_data = MyImageFolder(root = test_dir,
                                transform = test_transforms)
valid_data = MyImageFolder(root = val_dir,
                                transform = test_transforms)
train_iterator = data.DataLoader(train_data, 
                                shuffle = True,
                                drop_last = True,
                                batch_size = BATCH_SIZE) 
valid_iterator = data.DataLoader(valid_data, drop_last = True,
                                batch_size = BATCH_SIZE)
test_iterator = data.DataLoader(test_data, drop_last = True,
                                batch_size = BATCH_SIZE)     


In [ ]:
LEARNING_RATE = 1e-6
EPOCHS = 400
WEIGHT_DECAY=1e-3

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50')
# change output dimension to what we need
IN_FEATURES = model.fc.in_features 
OUTPUT_DIM = 2
model.fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
model.load_state_dict(torch.load('basic-model-hcb-pretrain.pt'))
fc1=nn.Linear(IN_FEATURES, 32)
fc2=nn.Linear(32,OUTPUT_DIM)
model.fc = nn.Sequential(fc1, fc2)

# criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1/256,1/410]).to(device))
model = model.to(device)
criterion = criterion.to(device)

params = [
          {'params': model.conv1.parameters(), 'lr': LEARNING_RATE / 20},
          {'params': model.bn1.parameters(), 'lr': LEARNING_RATE / 20},
          {'params': model.layer1.parameters(), 'lr': LEARNING_RATE / 10},
          {'params': model.layer2.parameters(), 'lr': LEARNING_RATE / 10},
          {'params': model.layer3.parameters(), 'lr': LEARNING_RATE / 4},
          {'params': model.layer4.parameters(), 'lr': LEARNING_RATE / 4},
          {'params': model.fc.parameters(), 'lr': LEARNING_RATE}
         ]
optimizer = optim.Adam(params, lr = LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 'min', factor=0.8, patience=5, min_lr=1e-30)

# some functions for modelling
def calculate_accuracy(y_pred, y):
    with torch.no_grad():
        batch_size = y.shape[0]
        _, pred = torch.max(y_pred, 1)
        correct = pred.eq(y).sum().item()
        #correct = correct[:1].view(-1).float().sum(0, keepdim = True)
        acc = correct / batch_size
    return acc

def train(model, iterator, optimizer, criterion, scheduler, device):  
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for (xy, _id) in iterator:
        x, y = xy
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()      
        y_pred = model(x)
        loss = criterion(y_pred, y)
        acc = calculate_accuracy(y_pred, y)
        loss.backward()
        optimizer.step()
        # scheduler.step()
        epoch_loss += loss.item()
        epoch_acc += acc
        
    epoch_loss /= len(iterator)
    epoch_acc /= len(iterator)
    return epoch_loss, epoch_acc

def evaluate(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for (xy, _id) in iterator:
            x, y = xy
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            acc = calculate_accuracy(y_pred, y)
            epoch_loss += loss.item()
            epoch_acc += acc
        
    epoch_loss /= len(iterator)
    epoch_acc /= len(iterator)   
    return epoch_loss, epoch_acc
    
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs  

In [ ]:
# train the basic model
print(BATCH_SIZE, LEARNING_RATE, WEIGHT_DECAY)
best_valid_loss = float('inf')
for epoch in range(EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, scheduler, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    scheduler.step(valid_loss)     
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), save_name)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} / {EPOCHS} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Valid Loss: {valid_loss:.3f}|')

In [ ]:
model.load_state_dict(torch.load(save_name))
test_loss, test_acc = evaluate(model, test_iterator, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:6.2f}% ')